<a href="https://colab.research.google.com/github/Kim-TaeKyoung/a/blob/main/0_1_Video_to_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python

In [ ]:
# -*- coding: utf-8 -*-
import glob
import json
import sys
import cv2
import os
import itertools
import math

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_path = "/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/raw/kdx-dataset/train/*"
test_path = "/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/raw/kdx-dataset/test/*"

video_file_train_list = glob.glob(test_path) #파일 경로 리스트형태로 반환
video_file_train_list_mp4 = [file for file in video_file_train_list if file.endswith(".mp4")] #mp4로 끝나는 파일 경로를 리스트에 저장
video_file_train_list_mp4 = sorted(video_file_train_list_mp4) 
# /content/drive/MyDrive/Colab/AI_ChampionShip/dataset/raw/kdx-dataset/test/sample_fire_scenes.json
with open('/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/raw/kdx-dataset/test/sample_fire_scenes.json', 'r') as f:
  json_data = json.load(f) 

a = list(json_data) # [a.mp4, b.mp4 ---- z.mp4]

In [ ]:
def check_fire(i):
  DIR1 = "/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/test/fire/"
  DIR2 = "/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/test/nonfire/"
  train_video = video_file_train_list_mp4[i] # mp4 path
  json_video = a[i] # mp4 json
  id = json_data[json_video]['id']
  print('\t')

  vidcap = cv2.VideoCapture(train_video)
  if not vidcap.isOpened():
    print("could not open :", train_video.split('/')[-1])
    exit(0)
  print('{}번 영상입니다'.format(id))
  second = 1
  width = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)
  height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)
  fps = round(vidcap.get(cv2.CAP_PROP_FPS)).       # 30프레임으로 맞추기 
  multiplier = fps * second                        # 1초마다 캡쳐  
  print('Video Resolution: %d, %d' % (width, height))
  print('Frame Count:', int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)))
  print('FPS:', fps)
  print('=='*10)
  
  count = 0
  n = 0 
  k = len(json_data[json_video]['scenes'])         # 화재영상구간 길이(=개수)

  if vidcap.isOpened():  
    while True:
      frameId = int(round(vidcap.get(1)))
      ret, image = vidcap.read()
      if ret:
        if frameId % multiplier < 1:
          if json_data[json_video]['scenes']:
            if n < k:                             # 화재 영상 구간 개수만큼 반복  ex) n = 0 : 첫번쨰 영상구간
              if count >= json_data[json_video]['scenes'][n][0] and count <= json_data[json_video]['scenes'][n][1]:  # 화재영상 구간 
                cv2.imwrite(DIR1 + "[%s]%d - fire.jpg" % (id,count),image)                                           # 경로 + 순서 + fire 
                print("saved image %d.jpg - fire detection" % count) 
                count += 1
                

              elif count == json_data[json_video]['scenes'][n][1] + 1: # 화재영상 마지막시간 바로 다음 지점(초)
                cv2.imwrite(DIR2 + "[%s]%d.jpg" % (id,count),image) 
                print("saved image %d.jpg" % count)
                count += 1
                n += 1 

              else:
                cv2.imwrite(DIR2 + "[%s]%d.jpg" % (id,count),image) # 화재영상 구간들 사이에 화재장면이 없는 지점
                print("saved image %d.jpg" % count)
                count += 1

            else:
              cv2.imwrite(DIR2 + "[%s]%d.jpg" % (id,count),image) # 화재가 없는 영상 지점 
              print("saved image %d.jpg" % count)
              count += 1
          else:
            continue
  
      if not ret: # 새로운 프레임을 못받아 왔을 때 braek
        break

      # if cv2.waitKey(10) == 27:                    
      #   break


In [ ]:
for i in  a:  
  check_fire(json_data[i]['id']) # -> i로  